In [1]:
import pandas as pd
import os
import time
import requests
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re

def download_records_after(url, date, page_size=10000):
    data = []
    page = 1   
    url_params = f'?filter=record_date:gt:{date}&page[size]={page_size}'
    response = requests.get(url + url_params)
    if response.status_code == 200:
        result_json = response.json()
        data.extend(result_json['data'])

        while True:
            if result_json['links']['next'] is None:
                break
            else:
                response = requests.get(url + result_json['links']['next'])
                if response.status_code == 200:
                    result_json = response.json()
                    data.extend(result_json['data'])
                    page += 1
                    print(f'page {page} of {result_json["meta"]["total-pages"]}')
                    time.sleep(1)
                else:
                    print(f'status_code: {response.status_code}')
                    break
    else:
        print(f'status_code: {response.status_code}')

    return pd.DataFrame(data)

def update_records(url, start_date='1900-01-01', page_size=10000, path=None, lookback=2):
    if path is None:
        ls = url.split('/')[3:]
        path = '-'.join(ls).replace('_', '-') + '.pkl'
    if os.path.isfile(path):
        print(f'Found {path}. Importing.')
        df = pd.read_pickle(path)
        recent_record_date = df['record_date'].unique()[-lookback]
        print(f'recent_record_date: {recent_record_date} lookback: {lookback}')
        new_records = download_records_after(url, recent_record_date, page_size)
        df = df[df['record_date'] <= recent_record_date]
        df = pd.concat([df, new_records], ignore_index=True)
        df.to_pickle(path)
        return df
    else:
        recent_record_date = start_date
        print(f'Using recent_record_date: {recent_record_date}')
        df = download_records_after(url, recent_record_date, page_size)
        df.to_pickle(path)
        return df

def upcoming_issuances_table(days_ahead=120):
    api_url = 'https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query'
    
    df = update_records(api_url, start_date='1900-01-01', page_size=10000, path=None, lookback=2)

    df['issue_date'] = pd.to_datetime(df['issue_date'])
    df['offering_amt'] = pd.to_numeric(df['offering_amt'], errors='coerce')

    # Filter future issuances
    today = pd.Timestamp.today().normalize()
    cutoff = today + pd.Timedelta(days=days_ahead)
    upcoming = df[(df['issue_date'] >= today) & (df['issue_date'] <= cutoff)].copy()

    if upcoming.empty:
        print(f"No upcoming issuances in the next {days_ahead} days.")
        return

    # Convert to billions
    upcoming['offering_amt_bil'] = upcoming['offering_amt'] / 1e9

    upcoming = upcoming[['issue_date', 'security_type', 'security_term_week_year', 'cusip', 'offering_amt_bil']]
    upcoming = upcoming.sort_values('issue_date')

    summary = upcoming.groupby('security_term_week_year')['offering_amt_bil'].sum().reset_index()
    summary = summary.rename(columns={'offering_amt_bil': 'Total Issuance (Billion $)'})

    # Sort by numeric tenor
    def parse_tenor(term):
        num = re.findall(r'\d+\.?\d*', term)
        unit = 'Y'
        if 'Week' in term:
            unit = 'W'
        elif 'Month' in term:
            unit = 'M'
        elif 'Year' in term:
            unit = 'Y'
        return float(num[0]) if num else 0, unit

    summary['sort_key'] = summary['security_term_week_year'].apply(lambda x: parse_tenor(x)[0])
    summary['unit'] = summary['security_term_week_year'].apply(lambda x: parse_tenor(x)[1])
    # Convert everything to weeks for sorting
    summary['sort_weeks'] = summary.apply(lambda row: row['sort_key'] * (1 if row['unit']=='W' else 4 if row['unit']=='M' else 52), axis=1)
    summary = summary.sort_values('sort_weeks')

    # --- Create Subplots ---
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=(f"Upcoming Treasury Issuances (Next {days_ahead} Days, Billion USD)",
                        f"Total Issuance by Term (Next {days_ahead} Days, Billion USD)"),
        vertical_spacing=0.15,
        specs=[[{"type": "table"}], [{"type": "table"}]]
    )

    fig.add_trace(go.Table(
        header=dict(
            values=["Issue Date", "Security Type", "Term", "CUSIP", "Offering (Billion $)"],
            fill_color='black', font=dict(color='white', size=14), align='center'
        ),
        cells=dict(
            values=[
                upcoming['issue_date'].dt.strftime('%Y-%m-%d'),
                upcoming['security_type'],
                upcoming['security_term_week_year'],
                upcoming['cusip'],
                upcoming['offering_amt_bil'].round(2)
            ],
            fill_color='rgb(30,30,30)',
            font=dict(color='white', size=12),
            align='center'
        )
    ), row=1, col=1)

    # Summary table
    fig.add_trace(go.Table(
        header=dict(
            values=["Term", "Total Issuance (Billion $)"],
            fill_color='black', font=dict(color='white', size=14), align='center'
        ),
        cells=dict(
            values=[
                summary['security_term_week_year'],
                summary['Total Issuance (Billion $)'].round(2)
            ],
            fill_color='rgb(30,30,30)',
            font=dict(color='white', size=12),
            align='center'
        )
    ), row=2, col=1)

    fig.update_layout(template='plotly_dark',
                      height=900)
    fig.show()

    return upcoming, summary

upcoming, summary = upcoming_issuances_table(days_ahead=120)

Found services-api-fiscal-service-v1-accounting-od-auctions-query.pkl. Importing.
recent_record_date: 2025-08-29 lookback: 2
